In [1]:
import os
os.environ["PATH"] += os.pathsep + 'C:/Users/anura/anaconda3/Library/bin/graphviz'
import sys
import networkx as nx
import numpy as np
import random
import pickle
import sys
from numba import jit
import elfi
import sklearn as sk
import scipy
from scipy.spatial import distance
import pandas as pd
import sys
print(sys.version)

3.7.9 (default, Aug 31 2020, 12:42:55) 
[GCC 7.3.0]


In [2]:
seed = 20170530  # this will be separately given to ELFI
np.random.seed(seed)
N = 100 # samples for rejection sampling
networktype = 'pref' #pref, smallworld, grid, ER, korea1, korea2, ckm
globalThreshold = .95
activation_ci = 0.15
cascadeParameter = 0.5
# nAgents = 4**2
# nAgents = 10**2
# sidelength = int(nAgents**.5) #10^2 = 100, 32^2 = 1024
#haltMin = .6 # minimum % of nodes active
#haltMax = .7 # maximum % of nodes active

# pRewire = .1 # proportion of edges to rewire
# numDatasets = 100 # number of data sets to generate
# saveData = True #save the output to file?


In [3]:
print(nx.__version__)
#parameters of the script
path = '../data/icpsr/DS0001/paluck-edgelist-seed.csv'
edgelist = pd.read_csv(path)
G = nx.from_pandas_edgelist(edgelist, source='ID', target='PEERID')
print(nx.info(G))
print(nx.number_of_nodes(G))
print(f'connected?\t{nx.is_connected(G)}')
print(f'# of connected components:\t{nx.number_connected_components(G)}')

components = nx.connected_components(G)
sglist = [G.subgraph(c) for c in nx.connected_components(G)]

gmat = []
for g in sglist:
    gmat.append(nx.to_numpy_matrix(g, dtype=np.float))


2.5
Name: 
Type: Graph
Number of nodes: 23882
Number of edges: 132054
Average degree:  11.0589
23882
connected?	False
# of connected components:	57


In [4]:
print(len(sglist))

57


In [5]:
#for i in range(len(sglist)):
#    print('# of nodes in {i}th component is  - ', str(np.mean(gmat[i])))
seeds = pd.read_csv("../data/icpsr/DS0001/paluck-seed.csv")
seeds['ID'] = ((seeds['SCHIDW2'] * 1000) + pd.to_numeric(seeds['ID'], errors='coerce'))

In [6]:
es = pd.read_csv("../data/icpsr/DS0001/paluck-endstate.csv")
es['ID'] = ((es['SCHIDW2'] * 1000) + pd.to_numeric(es['ID'], errors='coerce'))

 /home/gary/.conda/envs/elfi/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3156: DtypeWarning:Columns (238,242,244,245,246,247,248,249,382) have mixed types.Specify dtype option on import or set low_memory=False.


In [7]:
def genNet(n, k=4, pRewire=.1, type='grid'):
    # create net
    if type == 'grid': #wrap the grid
        net = nx.grid_2d_graph(int(n**.5), int(n**.5), periodic=True)
        #net = nx.grid_2d_graph(3, 3, periodic=True)
        #net = nx.grid_2d_graph(2, 2, periodic=False)
        #plotNet(net)
        #print(len(net.edges()))
        #assert(0)
        # rewire
        numRewired = 0
        #while numRewired < (pRewire * nx.number_of_nodes(net)):
        while numRewired < 1:
            tries = 0
            while tries < 100:
                tries = tries + 1
                #print([numRewired, pRewire * nx.number_of_nodes(net), tries])
                v1 = random.choice(net.nodes())
                v2 = random.choice(net.nodes())
                if not( net.has_edge(v1,v2) or v1==v2 or len(net.neighbors(v1)) <= 1): #net.neighbors is sometimes (often?) a blank set, changed so v1 needs 2 nb
                    #print net.neighbors(v1)
                    break
            v1Neighbors = net.neighbors(v1)
            #print v1Neighbors
            #print v1
            #print v2
            #print(len(net.edges()))
            tobeDeleted = random.choice(v1Neighbors)
            net.remove_edge(v1, tobeDeleted)
            #print(len(net.edges()))
            #print([v1, tobeDeleted, v2])
            net.add_edge(v1, v2)
            numRewired = numRewired + 1
            #plotNet(net)
            #assert(0)
        return net, nx.to_numpy_matrix(net, dtype=np.float)

    elif type == 'smallworld':
        #net = nx.connected_watts_strogatz_graph(n, k, .15)
        net = nx.connected_watts_strogatz_graph(n, k, pRewire)
        return net, nx.to_numpy_matrix(net, dtype=np.float)  
    elif type == 'pref':
        net = nx.barabasi_albert_graph(n, 2)
        return net, nx.to_numpy_matrix(net, dtype=np.float)
    elif type == 'ER':
        #net = nx.erdos_renyi_graph(n, .006)
        targetDegree = 4.
        nEdgesPossible = ((n*n)-n)/2.
        pEdge = (n * targetDegree) / (2. * nEdgesPossible)
        assert(pEdge <= 1)

        # spare networks will likely be disconnected, so try a bunch
        tries = 100
        while tries > 0:
            net = nx.erdos_renyi_graph(n, pEdge)
            if nx.number_connected_components(net) > 1:
                tries = tries - 1
            else:
                break
        return net, nx.to_numpy_matrix(net, dtype=np.float)

In [8]:
def testThresh(agents, agents_ini, mn, mx):
    
    if np.mean(agents) == 0:
        print("stopping because no nodes are activated in the initial state")
        return agents, False
    
    if np.mean(agents) >= mx:
        print("Bad data discarding the dataset because activation reached ", np.mean(agents))
        agents = agents_ini
        return agents, False
    
    if np.mean(agents) <= mn:
        return agents, False
    
    
    if np.mean(agents) > mn and np.mean(agents) < mx:
        print("Achieved stopping condition and stopping diffusion")
        return agents, True
        

    return False


In [9]:
def ltProp(agents, adjMatrix, nAgents, avgDegree=0, haltMin=.49, haltMax=.51, rs = None):
    # init some stuff for numba
    thresholds = np.zeros_like(nAgents)
    inp = np.zeros_like(nAgents)
    step = 0
    numNeighbors = np.zeros_like(nAgents)
    prevMean = 0.
    liveEdges = np.zeros_like(adjMatrix)
    pInfect = np.zeros_like(adjMatrix)
    flips = np.zeros_like(adjMatrix)
    

    if rs is None:
        thresholds = globalThreshold * np.random.random((1, nAgents))
        print("no seed passed")
    else:
        thresholds = globalThreshold * rs.random((1, nAgents))
    
    numNeighbors = np.sum(adjMatrix, axis=0)
    prevMean = -1
    step = 1
#     print("stopping conditions: ", haltMin, haltMax)
    print("running LT Prop")

    agents_ini = agents
    
    loop_manager = False
    
    while not loop_manager  and (np.mean(agents) > prevMean):
        
        agents, loop_manager = testThresh(agents, agents_ini, haltMin, haltMax)
        print("state activation: ", prevMean)

        #while np.mean(agents) > prevMean:
        prevMean = np.mean(agents)
        inp = np.true_divide(np.dot(agents, adjMatrix), numNeighbors)
        agents = np.logical_or(agents, (inp >= thresholds)).astype(int)

        step = step + 1

    print("state activation: ", prevMean)

    #print('LT-proportional stopped at step ' + str(step) + ' '+ str(np.mean(agents)))
    return agents

In [10]:
def ltAbs(agents, adjMatrix, nAgents, avgDegree=0, haltMin=.49, haltMax=.51, rs = None):
    # init some stuff for numba
    thresholds = np.zeros_like(nAgents)
    inp = np.zeros_like(nAgents)
    step = 0
    numNeighbors = np.zeros_like(nAgents)
    prevMean = 0
    liveEdges = np.zeros_like(adjMatrix)
    pInfect = np.zeros_like(adjMatrix)
    flips = np.zeros_like(adjMatrix)

    # this controls the thresholds/pInfects for all contagion types
    
    if rs is None:
        thresholds = np.random.randint(low=1, high=round(avgDegree*globalThreshold), size=(1, nAgents))
        print("no seed passed")
    else:
        thresholds = rs.randint(low=1, high=round(avgDegree*globalThreshold), size=(1, nAgents))
        print("seed passed")
    
    numNeighbors = np.sum(adjMatrix, axis=0)
    prevMean = -1
    step = 1
    print("running LT Abs")

    agents_ini = agents
    loop_manager = False
    while not loop_manager and (np.mean(agents) > prevMean):
        
        agents, loop_manager = testThresh(agents, agents_ini, haltMin, haltMax)
        print("state activation: ", prevMean)

        prevMean = np.mean(agents)
        inp = np.dot(agents, adjMatrix)
        agents = np.logical_or(agents, (inp >= thresholds)).astype(int)
        step = step + 1

    print("state activation: ", prevMean)

    # print('LT-absolute stopped at step ' + str(step) + ' '+ str(np.mean(agents)))

    return agents

In [11]:
def IC(agents, adjMatrix, nAgents, avgDegree=0, haltMin=.49, haltMax=.51, rs = None):
    # init some stuff for numba
    thresholds = np.zeros_like(nAgents)
    inp = np.zeros_like(nAgents)
    step = 0
    numNeighbors = np.zeros_like(nAgents)
    prevMean = -1
    liveEdges = np.zeros_like(adjMatrix)
    pInfect = np.zeros_like(adjMatrix)
    flips = np.zeros_like(adjMatrix)

        
    numNeighbors = np.sum(adjMatrix, axis=0)
    #prevMean = -1
    step = 1
    
    # calculate each edges probability of allowing infection
    
    # determine 'live' and 'blocked' edges
    
    print("running IC")
    if rs is None:
        pInfect = np.multiply(adjMatrix, np.random.random(adjMatrix.shape))
        flips = np.random.random(pInfect.shape)
    else:
        pInfect = np.multiply(adjMatrix, rs.random(adjMatrix.shape))
        flips = rs.random(pInfect.shape)
        
    pInfect = np.multiply(0.5, pInfect)
    liveEdges = flips < pInfect
    
    agents_ini = agents
        
    loop_manager = False
    print("starting loop manager")
    count = 40
    while not loop_manager and (np.mean(agents) > prevMean):
        agents, loop_manager = testThresh(agents, agents_ini, haltMin, haltMax)
        print("state activation: ", prevMean)

        prevMean = np.mean(agents)
        inp = np.dot(agents, liveEdges)
        agents = np.logical_or(agents, inp).astype(int)
        step = step + 1
        
        if count >= 40:
            loop_manager = True
        count += 1
        
    print("state activation: ", prevMean)

    # print('LT-absolute stopped at step ' + str(step) + ' '+ str(np.mean(agents)))

    return agents

In [12]:
def generateGraph(net, adjMatrix, pp, batch_size = 1, random_state=None):
    gg = []

    while True:
        nAgents = nx.number_of_nodes(net)
        agents = np.zeros((1, nAgents))
        nodes = nx.nodes(net)
        seed_nodes = []
        inx = []
        count = 0
        for n in nodes:
            if n in seeds['ID'].tolist():
                seed_nodes.append(n)
                inx.append(count)
            count += 1
        
        agents[:,inx] = 1

        es_nodes = []
        es_inx = []
        count_es = 0
        num_es = 0
        for n in nodes:
            if n in es['ID'].tolist():
                es_nodes.append(n)
                es_inx.append(count_es)
                num_es += 1
            count_es += 1
    
        p_activation = num_es/count_es
        print("final_activation: ", p_activation)
        print("Adjaecency Matrix: ", adjMatrix.shape)
        # seed neighbors of seeds
        print("zero initialized agents: ", agents[agents == 1].shape)
#         for i in range(1):
#             agents = np.logical_or(agents, np.dot(agents, adjMatrix)).astype(float) #all neighbors

        avgDegree = 2*net.number_of_edges() / float(net.number_of_nodes())
        print("initially activated agents: ", agents[agents == 1].shape)
#         if pp >= 0.5:
#             agents_type = ltProp(agents, adjMatrix, nAgents, avgDegree=avgDegree, 
#                                  haltMin = max(0,p_activation-activation_ci), 
#                                  haltMax = min(p_activation+activation_ci,1), 
#                                  rs = random_state)

#         else:
#             agents_type = ltAbs(agents, adjMatrix, nAgents, avgDegree=avgDegree, 
#                                 haltMin = max(0,p_activation-activation_ci), 
#                                 haltMax = min(p_activation+activation_ci,1), 
#                                 rs = random_state)

        t = 1/3
    
        if pp <= t:
            agents_type = IC(agents, adjMatrix, nAgents, avgDegree=avgDegree, 
                                 haltMin = max(0,p_activation-activation_ci), 
                                 haltMax = min(p_activation+activation_ci,1), 
                                 rs = random_state)

        elif pp > t and pp <= 2*t:
            agents_type = ltProp(agents, adjMatrix, nAgents, avgDegree=avgDegree, 
                                 haltMin = max(0,p_activation-activation_ci), 
                                 haltMax = min(p_activation+activation_ci,1), 
                                 rs = random_state)

        else:
            agents_type = ltAbs(agents, adjMatrix, nAgents, avgDegree=avgDegree, 
                                haltMin = max(0,p_activation-activation_ci), 
                                haltMax = min(p_activation+activation_ci,1), 
                                rs = random_state)

            
        #if not testThresh(agents_type, haltMin, haltMax):
         #   print('bad data, LTabs1:\t'+str(np.mean(agents_type)))
       #     continue
        gparam = {}
        gparam['init'] = net
        gparam['graph'] = agents_type
        
        gg.append(gparam['graph'])
        if len(gg) >= batch_size:
            break
    
    print("final graph: ", gg[0])
    return gg[0]



In [13]:
#n,a = getGraphFromEdgelist(path)
net_s = sglist[0]
adjMat_s = gmat[0]
print(adjMat_s[adjMat_s == 1].shape)

(1, 2960)


In [14]:
def compileG(ll, batch_size = 1, random_state=None):
    print("probability paramaeter: ", ll[0])
    return generateGraph(net_s, adjMat_s, ll[0], batch_size, random_state)


In [15]:
haye mera dilnAgobs = nx.number_of_nodes(net_s)
agobs = np.zeros((1, nAgobs))
nodeobs = nx.nodes(net_s)
es_nodeobs = []
es_inxobs = []
count_esobs = 0
num_esobs = 0

for n in nodeobs:
    if n in es['ID'].tolist():
        es_nodeobs.append(n)
        es_inxobs.append(count_esobs)
        num_esobs += 1
    count_esobs += 1

agobs[:,es_inxobs] = 1
print(num_esobs)
print(count_esobs)

124
272


In [16]:
gr_obs = np.matrix(agobs).astype(int)


In [17]:
prop_prob = elfi.Prior(scipy.stats.uniform, 0.0, 1.0)
print(type(prop_prob))

<class 'elfi.model.elfi_model.Prior'>


In [18]:
print(gr_obs.shape[1])

272


In [19]:
# Y = elfi.Simulator(generateGraph, prop_prob, observed = gr_obs)
Y = elfi.Simulator(compileG, prop_prob, observed = gr_obs)

In [20]:
print(Y)

Y


In [21]:
def yMetric(x, i = 0):
    return x[0,i]


In [22]:
def eucMultiArgs(X, Y):
    dist = np.linalg.norm(X - Y) 
    return np.array([dist])

In [23]:
ret = []
for i in range(gr_obs.shape[1]):
    st = str(i)
    var_s = ''.join(['s',st])
    ret.append(var_s)
    com = var_s + ' = ' + 'elfi.Summary(yMetric, Y, '+st+')'
    exec(com)
    

Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Py

Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Py

Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Py

Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Py

Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Py

Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Py

Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Py

Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.
Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Py

In [24]:
dcom = "d=elfi.Distance('euclidean'"
# len_run = min(250, gr_obs.shape[1])
for i in range(gr_obs.shape[1]):
    if i == 0:
        dcom = dcom + ',s' + str(i)
    else:
        dcom = dcom + ',s' + str(i)
dcom = dcom + ')'
print(dcom)
exec(dcom)
# d = elfi.Distance('euclidean',s7)
# d = elfi.Distance('euclidean',s0, s1, s2, s3, s4, s5, s6)

Automatic name inspection failed, using a random name instead. This may be caused by using an interactive Python shell. You can provide a name parameter e.g. elfi.Prior('uniform', name='nodename') to suppress this warning.


d=elfi.Distance('euclidean',s0,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21,s22,s23,s24,s25,s26,s27,s28,s29,s30,s31,s32,s33,s34,s35,s36,s37,s38,s39,s40,s41,s42,s43,s44,s45,s46,s47,s48,s49,s50,s51,s52,s53,s54,s55,s56,s57,s58,s59,s60,s61,s62,s63,s64,s65,s66,s67,s68,s69,s70,s71,s72,s73,s74,s75,s76,s77,s78,s79,s80,s81,s82,s83,s84,s85,s86,s87,s88,s89,s90,s91,s92,s93,s94,s95,s96,s97,s98,s99,s100,s101,s102,s103,s104,s105,s106,s107,s108,s109,s110,s111,s112,s113,s114,s115,s116,s117,s118,s119,s120,s121,s122,s123,s124,s125,s126,s127,s128,s129,s130,s131,s132,s133,s134,s135,s136,s137,s138,s139,s140,s141,s142,s143,s144,s145,s146,s147,s148,s149,s150,s151,s152,s153,s154,s155,s156,s157,s158,s159,s160,s161,s162,s163,s164,s165,s166,s167,s168,s169,s170,s171,s172,s173,s174,s175,s176,s177,s178,s179,s180,s181,s182,s183,s184,s185,s186,s187,s188,s189,s190,s191,s192,s193,s194,s195,s196,s197,s198,s199,s200,s201,s202,s203,s204,s205,s206,s207,s208,s209,s210,s211,s212,s213,s214,s215,s2

In [25]:
#d=elfi.Distance(eucMultiArgs,s0,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21,s22,s23,s24,s25,s26,s27,s28,s29,s30,s31,s32,s33,s34,s35,s36,s37,s38,s39,s40,s41,s42,s43,s44,s45,s46,s47,s48,s49,s50,s51,s52,s53,s54,s55,s56,s57,s58,s59,s60,s61,s62,s63,s64,s65,s66,s67,s68,s69,s70,s71,s72,s73,s74,s75,s76,s77,s78,s79,s80,s81,s82,s83,s84,s85,s86,s87,s88,s89,s90,s91,s92,s93,s94,s95,s96,s97,s98,s99,s100,s101,s102,s103,s104,s105,s106,s107,s108,s109,s110,s111,s112,s113,s114,s115,s116,s117,s118,s119,s120,s121,s122,s123,s124,s125,s126,s127,s128,s129,s130,s131,s132,s133,s134,s135,s136)

In [26]:
# rej = elfi.Rejection(d, batch_size=1, seed=seed)
rej = elfi.Rejection(d, batch_size=1)

In [27]:
# You can give the sample method a `vis` keyword to see an animation how the prior transforms towards the
# posterior with a decreasing threshold.
%time result = rej.sample(N, quantile=0.1)

probability paramaeter:  0.25223242053512784
final_activation:  0.45588235294117646
Adjaecency Matrix:  (272, 272)
zero initialized agents:  (19,)
initially activated agents:  (19,)
running IC
starting loop manager
state activation:  -1
state activation:  0.06985294117647059
final graph:  [[0 1 0 0 1 0 1 1 0 1 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 1 0
  0 0 0 0 1 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0
  1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0
  0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 1 1 0 0 1 0 1 1 0 1 0 0 0 0 0 0 0
  1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 1
  0 0 0 0 1 0 0 0 0 1 0 1 1 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 1 1 0 0 1 1 0 0
  0 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0 0 1 0 1]]
probability paramaeter:  0.25682396714894007----------------] 0.1% Complete
final_activation:  0.45588235294117646
Adjaecency Matrix:  (272, 272)
ze

final_activation:  0.45588235294117646
Adjaecency Matrix:  (272, 272)
zero initialized agents:  (19,)
initially activated agents:  (19,)
running LT Prop
state activation:  -1
state activation:  0.06985294117647059
state activation:  0.13970588235294118
state activation:  0.20955882352941177
state activation:  0.2536764705882353
Achieved stopping condition and stopping diffusion
state activation:  0.3014705882352941
state activation:  0.38235294117647056
final graph:  [[0 1 0 1 0 0 0 0 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0
  1 1 1 1 1 1 1 1 0 1 0 1 1 1 0 1 1 0 1 0 0 0 1 0 0 0 0 0 0 0 1 0 0 1 1 0
  0 0 1 0 1 1 1 0 1 0 1 0 0 1 0 0 1 0 1 0 0 0 1 0 0 1 1 0 1 0 1 1 1 0 1 0
  0 0 0 0 1 1 1 0 0 0 1 0 0 1 0 0 1 0 1 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 1 0
  0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 1
  0 1 0 1 0 1 0 0 0 0 0 1 0 0 1 0 0 1 1 0 1 0 1 0 1 1 1 0 0 1 1 0 1 0 1 1
  0 1 1 1 1 1 1 0 0 1 0 1 1 1 1 1 1 0 0 0 1 1 0 0 0 0 1 1 0 1 1 1 0 0 0 1
  1 0 1 0 

probability paramaeter:  0.42137737605846726----------------] 1.8% Complete
final_activation:  0.45588235294117646
Adjaecency Matrix:  (272, 272)
zero initialized agents:  (19,)
initially activated agents:  (19,)
running LT Prop
state activation:  -1
state activation:  0.06985294117647059
state activation:  0.16544117647058823
state activation:  0.22794117647058823
Achieved stopping condition and stopping diffusion
state activation:  0.2867647058823529
state activation:  0.34558823529411764
final graph:  [[0 1 1 0 0 1 1 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 1 1 1 0 0 1 0 0 1 0 1 1 1 1
  0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 1 0 1 0 0 1 1 0 1 0 0
  0 0 1 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 1 0 0 0 0 1 0 0
  0 0 1 0 1 0 0 0 1 1 1 1 0 0 0 0 1 0 0 1 0 0 1 1 0 0 0 0 0 0 1 0 0 1 0 0
  0 0 0 1 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 1 0 0 0 1 0 1 1 0 1 0 0 0 0 0
  1 1 0 0 1 0 1 0 0 0 0 1 1 0 1 1 1 0 1 0 1 1 1 0 0 1 0 0 0 0 1 0 0 1 0 1
  0 0 0 1 1 0 1 1 1 1 1 1 1 1 1 0 0 0 0 0 1 1 

probability paramaeter:  0.598144662961166------------------] 2.7% Complete
final_activation:  0.45588235294117646
Adjaecency Matrix:  (272, 272)
zero initialized agents:  (19,)
initially activated agents:  (19,)
running LT Prop
state activation:  -1
state activation:  0.06985294117647059
state activation:  0.15808823529411764
state activation:  0.23897058823529413
Achieved stopping condition and stopping diffusion
state activation:  0.3014705882352941
state activation:  0.34191176470588236
final graph:  [[1 1 1 0 0 1 0 1 0 1 0 0 1 0 1 1 0 1 1 1 0 0 1 1 1 1 1 1 0 0 1 0 0 0 1 0
  0 1 1 0 0 0 1 1 1 1 0 1 1 0 1 0 0 1 0 1 1 1 1 0 1 1 0 0 0 0 1 0 1 1 0 0
  0 0 0 0 1 1 1 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 1 1 0 1 1 1 0 0 0 0 0 0 0 0 1 1 1
  0 0 0 0 1 1 0 1 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 1 1 0
  0 1 0 1 0 0 0 1 0 1 1 0 0 0 0 1 1 0 0 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0
  1 0 0 0 1 0 1 0 0 1 1 1 1 0 0 1 0 0 0 1 0 1 

final_activation:  0.45588235294117646
Adjaecency Matrix:  (272, 272)
zero initialized agents:  (19,)
initially activated agents:  (19,)
seed passed
running LT Abs
state activation:  -1
state activation:  0.06985294117647059
state activation:  0.1323529411764706
Achieved stopping condition and stopping diffusion
state activation:  0.22794117647058823
state activation:  0.3272058823529412
final graph:  [[1 1 1 0 0 0 1 0 0 1 0 1 0 1 1 0 1 1 0 1 0 0 1 1 1 1 0 0 1 1 1 1 1 1 1 1
  0 1 0 0 1 1 0 1 1 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 1 0 1 1 0 0
  0 1 0 0 0 1 1 0 1 0 0 0 0 1 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 1 0 0 0 0 1 0 0
  0 0 0 0 1 0 0 1 0 1 1 0 0 1 1 0 1 0 1 1 1 1 1 0 1 0 1 1 0 0 1 0 0 1 1 0
  1 0 0 0 1 1 1 0 0 1 0 1 0 0 1 0 1 0 1 0 1 1 1 0 1 0 1 1 0 1 1 1 0 1 0 1
  0 0 0 1 1 1 1 0 0 1 0 1 1 1 0 0 0 1 0 0 0 1 1 1 0 1 1 0 0 1 1 1 1 1 1 1
  1 0 1 0 0 1 1 0 1 1 0 0 0 0 0 0 1 1 1 1]]
probability paramaeter:  0.566346

...

final_activation:  0.45588235294117646
Adjaecency Matrix:  (272, 272)
zero initialized agents:  (19,)
initially activated agents:  (19,)
running LT Prop
state activation:  -1
state activation:  0.06985294117647059
state activation:  0.14338235294117646
state activation:  0.1948529411764706
state activation:  0.22058823529411764
state activation:  0.23897058823529413
state activation:  0.2647058823529412
Achieved stopping condition and stopping diffusion
state activation:  0.3014705882352941
state activation:  0.3235294117647059
final graph:  [[1 1 0 0 1 0 1 0 0 1 1 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 0 1 0 0
  0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 0 1 0 0 1 1 0 0 0 0 0 0 1 0 0 1 0 0
  0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 0 1 0 1 1 1 1 0 0 1 0 0 0 1 0 0 0 0 1 0
  1 0 0 1 0 0 0 0 0 1 1 0 0 1 0 0 0 1 0 1 1 0 1 0 0 1 0 0 1 0 0 0 0 1 0 0
  0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0
  0 0 0 1 0 0 0 0 0 1 1 0 1 1 0 0 1 1 1 0 1 1 1 0 0 0 0 0 0 1 1 1 1 0 0 0
  1 1 0 

final_activation:  0.45588235294117646
Adjaecency Matrix:  (272, 272)
zero initialized agents:  (19,)
initially activated agents:  (19,)
running IC
starting loop manager
state activation:  -1
state activation:  0.06985294117647059
final graph:  [[0 1 0 0 0 1 0 0 1 1 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1
  0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 1 0 0
  1 0 0 1 0 1 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
  0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0
  0 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 1
  1 0 0 0 1 0 1 1 0 1 1 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0 1 1 0
  0 0 1 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 0 1]]
probability paramaeter:  0.2600027539722709===============--] 96.7% Complete
final_activation:  0.45588235294117646
Adjaecency Matrix:  (272, 272)
zero initialized agents:  (19,)
initially acti

state activation:  0.06985294117647059
final graph:  [[0 1 1 0 0 1 1 0 1 1 1 0 0 0 1 0 1 0 0 0 0 0 0 0 1 1 0 1 0 0 1 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0
  0 0 0 1 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0
  0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 1
  0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 1 1 0 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0
  1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 1 0 0 0 0 1 0 0 0 1
  1 1 1 1 1 0 0 1 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1]]
probability paramaeter:  0.06018882933844594==============--] 97.6% Complete
final_activation:  0.45588235294117646
Adjaecency Matrix:  (272, 272)
zero initialized agents:  (19,)
initially activated agents:  (19,)
running IC
starting loop manager
state activation:  -1
state activation:  0.06985294117647059
final graph:  [[1 1 0 0 0 1 0 1 0 1 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 1 0 0 0 1 1

probability paramaeter:  0.21870227555410404===============-] 98.5% Complete
final_activation:  0.45588235294117646
Adjaecency Matrix:  (272, 272)
zero initialized agents:  (19,)
initially activated agents:  (19,)
running IC
starting loop manager
state activation:  -1
state activation:  0.06985294117647059
final graph:  [[1 1 1 0 0 0 1 0 0 1 0 0 0 1 1 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 1 1 0
  0 0 0 0 1 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0
  1 0 0 1 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0
  0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1
  0 0 0 0 0 0 0 1 0 1 0 0 1 1 1 0 0 0 1 1 0 1 0 1 0 0 0 0 0 0 1 1 0 0 0 0
  1 0 0 1 0 0 0 0 0 0 1 0 0 0 1 1 1 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 1 0 1 1 0 1 1 1 0 0 0 0 0 0 0 1 0 0 1 1 1 0 1 0 0 1 0 0 0 1 0 0
  1 0 1 0 0 0 0 0 1 1 0 1 0 0 1 0 0 0 1 1]]
probability paramaeter:  0.017917216577663653==============-] 98.6% Complete
final_activation:  0.45588235294117646


probability paramaeter:  0.5713565856621856================-] 99.4% Complete
final_activation:  0.45588235294117646
Adjaecency Matrix:  (272, 272)
zero initialized agents:  (19,)
initially activated agents:  (19,)
running LT Prop
state activation:  -1
state activation:  0.06985294117647059
state activation:  0.13602941176470587
state activation:  0.1875
state activation:  0.23897058823529413
Achieved stopping condition and stopping diffusion
state activation:  0.2757352941176471
state activation:  0.3125
final graph:  [[0 1 0 0 0 1 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 1 0 0
  0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 1 0 0 0 1 1 1 1 0 0
  0 0 1 0 0 1 0 1 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0
  0 0 1 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 1 0 1 0 1 0 1 1 0 0 0 0 0 1
  0 0 1 0 0 0 0 0 0 0 1 1 0 1 1 0 0 1 0 1 0 0 0 1 0 1 0 1 0 1 1 0 0 1 1 1
  1 1 0 1 1 0 1 0 0 1 0 0 1 1 0 0 0 0 0 1 1 1 1 1 0 0 0 1 0 1 0 0 0 0 0 1
  1 0 0 1 1 0 0 0 0 1 1 1 1 0 0 

In [30]:
result.samples['prop_prob'].mean()


0.32239949755257313

In [29]:
print(result.samples['prop_prob'])
# print(len(result.samples['prop_prob']))

[0.48895212 0.55453656 0.09573104 0.15929742 0.64729801 0.28244888
 0.42456566 0.13291936 0.59522507 0.19040863 0.26430779 0.15818063
 0.29493815 0.55747761 0.12089016 0.55012217 0.36607953 0.53770286
 0.19477413 0.39009186 0.53212667 0.13813092 0.13926067 0.53542627
 0.32895746 0.37731071 0.26896755 0.23877631 0.59855167 0.10142344
 0.12175835 0.16775038 0.59814466 0.24044285 0.59939537 0.49502412
 0.11561896 0.01751258 0.32034958 0.47443052 0.80010184 0.884249
 0.50987707 0.35675715 0.40075348 0.01890992 0.16159251 0.08583117
 0.64933218 0.10796118 0.63008778 0.18353224 0.1897597  0.2098082
 0.32209587 0.1979459  0.6614572  0.62186062 0.25557493 0.26174308
 0.25094968 0.26884057 0.21745046 0.2132831  0.08989403 0.22658899
 0.35824764 0.44166002 0.51624152 0.04625755 0.55123997 0.10564633
 0.02925713 0.91833546 0.21691973 0.30287    0.00273092 0.30491976
 0.56188749 0.1563314  0.01779553 0.4105962  0.900511   0.36965477
 0.43687782 0.4446128  0.60767791 0.17983718 0.03388179 0.0435548